Importação de todos os pacotes que serão utilizados ao longo do modelo.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [2]:
#leitura dos dados
df_review = pd.read_csv("IMDB Dataset.csv")
#checagem do balanceamento
df_review.value_counts(subset='sentiment')

sentiment
negative    25000
positive    25000
Name: count, dtype: int64

Dataset já veio perfeitamente balanceado. Nesse caso, não há necessidade de tomar qualquer ação nesse sentido.  
De forma semelhante, os dados já estão limpos, não exigindo esse passo tampouco.

In [3]:
#separação dos dados em treino e teste, utilizando proporção 80/20
treino, teste = train_test_split(df_review, test_size=0.2, random_state=42)
treino_x, treino_y = treino['review'], treino['sentiment']
teste_x, teste_y = teste['review'], teste['sentiment']

treino_x: variáveis independentes (review/análises) utilizadas para treinar o modelo. 
treino_y: variáveis dependentes/labels (sentiment) que o modelo deve prever.  
teste_x: variáveis independentes que serão utilizadas para teste de precisão do modelo.  
teste_y: labels/etiquetas usadas para testar a precisão da previsão do modelo contra as categorias de fato

Para transformar a representação textual em vetores numéricos, utilizaremos a técnica de bag of words (BOW).  
A escolha se dá por: 1) a frequência das palavras nas análises importa, podendo ser indicador de seu sentimento; 2) a ordem em si das palavras possui menor relevância.  
A BOW será representada por Term Frequency, Inverse Document Frequency (TF-IDF), forma de ponderar o peso de cada palavra de um documento (no caso, análise individual) levando em conta sua presença no corpus total (o conjunto de documentos/análises).

In [4]:
tfidf = TfidfVectorizer(stop_words='english') #remoção de stop words/palavras vazias da língua inglesa
treino_x_vetor = tfidf.fit_transform(treino_x) #encontra parâmetros internos do modelo e os aplica, vetorizando o treino
teste_x_vetor = tfidf.transform(teste_x) #apenas vetoriza o teste para uso
treino_x_vetor

<40000x92692 sparse matrix of type '<class 'numpy.float64'>'
	with 3543198 stored elements in Compressed Sparse Row format>

Teste de diferentes modelos de aprendizagem supervisionada (classificação) para selecionar aquele com maior precisão. Testaremos Support Vector Machines (SVM), árvore de decisões, e regressão logística, comparando suas precisões médias (mean accuracy), F1 score, classification report e matriz de confusão.

In [5]:
#inserção dos dados nos algoritmos
#SVM
svc = SVC(kernel='linear')
svc.fit(treino_x_vetor, treino_y)


SVC(kernel='linear')

In [6]:
#Decision Tree
dec_tree = DecisionTreeClassifier()
dec_tree.fit(treino_x_vetor, treino_y)

DecisionTreeClassifier()

In [8]:
#Regressão logística
log_reg = LogisticRegression()
log_reg.fit(treino_x_vetor, treino_y)

LogisticRegression()

In [10]:
#Mean Accuracy
print(svc.score(teste_x_vetor, teste_y))
print(dec_tree.score(teste_x_vetor, teste_y))
print(log_reg.score(teste_x_vetor, teste_y))

0.8964
0.7228
0.8941


Como o resultado das árvores de decisão(0.7228) foi consideravelmente abaixo daqueles apresentados pelo SVM (0.8964) e regressão logística (0.8941), utilizaremos apenas esses dois últimos para os demais testes de comparação.

In [11]:
#F1 score
print(f1_score(teste_y, svc.predict(teste_x_vetor), 
         labels=['positive', 'negative'], average=None))
print(f1_score(teste_y, log_reg.predict(teste_x_vetor), 
         labels=['positive', 'negative'], average=None))

[0.89805156 0.89469404]
[0.89655172 0.89152924]


Scores bastante semelhantes/próximos, continuando uma pequena vantagem para o SVM.

In [12]:
#Classification report
print(classification_report(teste_y,
                            svc.predict(teste_x_vetor),
                            labels=['positive', 'negative']))
print(classification_report(teste_y,
                            log_reg.predict(teste_x_vetor),
                            labels=['positive', 'negative']))

              precision    recall  f1-score   support

    positive       0.89      0.91      0.90      5039
    negative       0.90      0.89      0.89      4961

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000

              precision    recall  f1-score   support

    positive       0.88      0.91      0.90      5039
    negative       0.91      0.88      0.89      4961

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



Novamente, uma pequena margem de vantagem para o SVM. Como sua accuracy foi maior que 0.895, foi arredondada para 0.9.  

In [14]:
#Confusion matrix
conf_mat_svc = confusion_matrix(teste_y,
                            svc.predict(teste_x_vetor),
                            labels=['positive', 'negative'])
conf_mat_logreg = confusion_matrix(teste_y,
                            log_reg.predict(teste_x_vetor),
                            labels=['positive', 'negative'])

In [16]:
print(conf_mat_svc)
print(conf_mat_logreg)

[[4563  476]
 [ 560 4401]]
[[4589  450]
 [ 609 4352]]


Outro resultado próximo. De forma geral, baseando-se nos resultados, o SVM apresenta melhor capacidade de previsão, ainda que ligeiramente. Entretanto, pode-se ponderar o uso da regressão logística em seu lugar, tendo em vista que, apesar de ser ligeiramente menos preciso, apresenta um tempo de processamento menor, em especial no treinamento do modelo (no caso, foi efetivamente mais de 70x mais rápido do que o SVM).